# Goals

* Test connecting and updating a GCP Cloud SQL database

In [1]:
import os
import warnings
import psycopg2
import pandas as pd
from pypika import Query, Table, Field, Column, Table

In [2]:
from dotenv import load_dotenv
load_dotenv()
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

# Connect

In [3]:
# Your GCP PostgreSQL connection parameters
# get home directory
host = os.path.join(os.path.expanduser("~"), "cloudsql", os.environ["GCP_SQL_DB_HOST"])

db_params = {
    'host': host,
    'database': os.environ["GCP_SQL_DB_NAME"],
    'user': os.environ["GCP_SQL_DB_USERNAME"],
    'password': os.environ["GCP_SQL_DB_PASSWORD"],
    'port': '5432',
    'connect_timeout': 10 
}

In [4]:
def get_db_connection():
    try:
        return psycopg2.connect(**db_params)
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

conn = get_db_connection()    

In [5]:
def execute_query(stmt, conn):
    try:
        with conn.cursor() as cur:
            cur.execute(str(stmt))
            conn.commit() 
    except psycopg2.errors.DuplicateTable as e:
        print(f"Table already exists: {e}")

In [41]:
# list tables in pypika
def list_tables_pypika():
    tables = Table('tables', schema='information_schema')
    query = Query.from_(tables).select('table_name').where(tables.table_schema == 'public')
    with conn.cursor() as cur:
        cur.execute(str(query))
        tables = cur.fetchall()
        return tables
list_tables_pypika()

[('srx_metadata',), ('srx_srr',), ('screcounter',)]

# Create tables

In [43]:
# SRX_metadata
stmt = Query \
    .create_table("srx_metadata") \
    .columns(
        Column("id", "SERIAL", nullable=False),
        Column("database", "VARCHAR(20)", nullable=False),
        Column("entrez_id", "INT", nullable=False),
        Column("srx_accession", "VARCHAR(20)"),
        Column("is_illumina", "VARCHAR(10)"),
        Column("is_single_cell", "VARCHAR(10)"),
        Column("is_paired_end", "VARCHAR(10)"),
        Column("is_10x", "VARCHAR(10)"),
        Column("tech_10x", "VARCHAR(20)"),
        Column("organism", "VARCHAR(60)"),
        Column("screcounter_status", "VARCHAR(20)"),
        Column("screcounter_log", "VARCHAR(2000)"),
    ) \
    .unique("database", "entrez_id") \
    .primary_key("id")

execute_query(stmt, conn)

In [8]:
# SRX_SRR
stmt = Query \
    .create_table("srx_srr") \
    .columns(
        Column("id", "SERIAL", nullable=False),
        Column("srx_accession", "VARCHAR(20)", nullable=False),
        Column("srr_accession", "VARCHAR(20)", nullable=False)
    ) \
    .unique("srx_accession", "srr_accession") \
    .primary_key("id")

execute_query(stmt, conn)

In [9]:
# scRecounter log
stmt = Query \
    .create_table("screcounter") \
    .columns(
        Column("id", "SERIAL", nullable=False),
        Column("sample_id", "VARCHAR(20)", nullable=False),
        Column("pipeline_version", "VARCHAR(10)", nullable=False),
        Column("run_id", "VARCHAR(30)", nullable=False),
        Column("task_name", "VARCHAR(20)", nullable=False),
        Column("task_exit_status", "VARCHAR(10)"),
        Column("log", "TEXT", nullable=False)
    ) \
    .primary_key("id")

execute_query(stmt, conn)

# Delete tables

> WARNING

In [42]:
# for table in ["srx_metadata", "srx_srr", "screcounter"]:
for table in ["srx_metadata"]:
    stmt = Query.drop_table(table)
    print(str(stmt))
    execute_query(stmt, conn)

DROP TABLE "srx_metadata"


# Insert data

In [33]:
srx_metadata = Table("srx_metadata")

q = Query.into(srx_metadata) \
    .columns('database', 'entrez_id', 'srx_accession', 'is_illumina', 'is_single_cell', 'is_paired_end', 'is_10x', 'tech_10x', 'organism') \
    .insert('sra', 35087715, 'SRX25994842', 'yes', 'yes', 'yes', 'yes', '3_prime_gex', 'human')

execute_query(q, conn)

# Query data

In [27]:
srx_metadata = Table("srx_metadata")
stmt = Query \
    .from_(srx_metadata) \
    .select("*") 
pd.read_sql(str(stmt), conn)

,id,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,is_10x,tech_10x,organism,screcounter_status,screcounter_log
0,2,sra,35087715,SRX25994842,yes,yes,yes,yes,3_prime_gex,human,None,None
1,3,sra,36178506,ERX11887200,yes,yes,yes,yes,3_prime_gex,mouse,None,None
2,5,sra,18060880,SRX13201194,yes,no,no,no,other,mouse,None,None
3,1,sra,36106630,SRX26727599,yes,no,yes,no,other,mouse,complete,test log
4,17,sra,30749595,SRX22716300,yes,yes,yes,yes,5_prime_gex,human,None,None


In [10]:
srx_srr = Table("srx_srr")
stmt = Query \
    .from_(srx_srr) \
    .select("*") 
pd.read_sql(str(stmt), conn)

,id,srx_accession,srr_accession
0,1,SRX26727599,SRR31350667
1,2,SRX23538581,SRR27876733
2,3,SRX23261451,SRR27592690
3,4,SRX23261451,SRR27592688
4,5,SRX23261451,SRR27592689
5,6,SRX23261451,SRR27592687
6,8,SRX25994842,SRR30571763
7,9,ERX11887200,ERR12511670
8,10,ERX11887200,ERR12511661


In [ ]:
tbl = Table("screcounter")
stmt = Query \
    .from_(tbl) \
    .select("*") 
pd.read_sql(str(stmt), conn)

/tmp/ipykernel_8985/2052542231.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(str(stmt), conn)


,id,sample_id,pipeline_version,run_id,task_name,task_exit_status,log


In [12]:
# filter to get unprocessed records
srx_metadata = Table("srx_metadata")
stmt = Query \
    .from_(srx_metadata) \
    .select("*") \
    .where((srx_metadata.processed != "complete") | (srx_metadata.processed.isnull())) \
    .where(srx_metadata.database == "sra")
pd.read_sql(str(stmt), conn)

/tmp/ipykernel_8985/2340524525.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(str(stmt), conn)


,id,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,is_10x,tech_10x,organism,processed


# Remove records

In [26]:
# remove from "srx_metadata" based on id column
record_id = 18
tbl = Table("srx_metadata")
stmt = Query \
    .from_(tbl) \
    .where(tbl.id == record_id) \
    .delete()
execute_query(stmt, conn)